## Description:
这个项目是答题卡识别检测任务，给定一个学生涂好的答题卡，通过检测，给出学生答错的错题以及完成最后的评分工作。

In [1]:
import numpy as np
import cv2

In [2]:
def cv_imshow(title, img):
    cv2.imshow(title, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
img_path = 'images/test_01.png'
img = cv2.imread(img_path)

In [4]:
cv_imshow('img', img)

In [10]:
contours_img = img.copy()

## 预处理
主要包括转成灰度图，高斯滤波，边缘检测， 轮廓检测

In [5]:
# 转成灰度图
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [13]:
cv_imshow('gray', gray)

In [6]:
# 高斯滤波
blured = cv2.GaussianBlur(gray, (5, 5), 0)

In [39]:
cv_imshow('blured', blured)

In [7]:
# 边缘检测
edged = cv2.Canny(blured, 75, 200)

In [41]:
cv_imshow('edged', edged)

In [8]:
# 轮廓检测
# 轮廓检测这里应该在边缘检测的结果上进行，才能锁定答题区域， 如果换成灰度图，这里检测不到答题卡的轮廓
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [11]:
_ = cv2.drawContours(contours_img, cnts, -1, (0, 0, 255), 3)

In [12]:
cv_imshow('contorus_img', contours_img)

In [30]:
len(cnts)

1

## 透视变换
有了答题卡轮廓，接下来，就是把答题卡轮廓的四个坐标拿到，然后对该区域做透视变换，让其铺满整个图像

In [13]:
dotCnt = None

# 这里对上面的轮廓进行一次筛选， 确保是答题卡的轮廓，因为有可能检测到里面的原形轮廓，另外一个就是最外面的答题卡轮廓目前并不是
# 很标准， 边缘是锯齿形的，所以要在看是否是最外面的轮廓之前，进行轮廓近似操作
if len(cnts) > 0:
    # 根据轮廓大小进行排序
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    for c in cnts:
        # 轮廓近似
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02*peri, True)
        
        # 保存四个顶点，为透视变换做准备
        if len(approx) == 4:
            dotCnt = approx
            break

In [54]:
dotCnt.shape

(4, 1, 2)

In [14]:
point_img = img.copy()
for point in dotCnt:
    cv2.circle(point_img, (point[0][0], point[0][1]), 10, (0, 0, 255), 4) # img, center, radius, color, thickness

cv_imshow('point_img', point_img)

In [15]:
def order_points(pts):
    # 一共四个点
    rect = np.zeros((4, 2), dtype='float32')
    
    # 按照顺序对应坐标0123分别是左上， 右上， 右下， 左下
    s = pts.sum(axis=1)  # 横纵坐标相加， 左上的之和最小，右下的之和最大
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)   # 纵坐标-横坐标， 右上的之差最小， 左下的之差最大
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [16]:
# 透视变换函数
def four_point_transform(img, pts):
    # 获取坐标点
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # 计算输入的w和h的值，方便定位新坐标
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # 变换后对应坐标位置
    dst = np.array([
        [0, 0], 
        [maxWidth-1, 0], 
        [maxWidth-1, maxHeight-1],
        [0, maxHeight-1]
    ], dtype='float32')
    
    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(img, M, (maxWidth, maxHeight))
    
    return warped

In [45]:
warped = four_point_transform(gray, dotCnt.reshape(4, 2))

In [18]:
cv_imshow('warped', warped)

## 圆圈的轮廓检测

In [19]:
# 在轮廓检测之前，先通过阈值把图像处理成黑白图像，这样后面找圆圈的轮廓才能更加清晰
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [116]:
cv_imshow('thresh', thresh)

In [20]:
# 检测每个圆圈轮廓
thresh_Countours = warped.copy()

In [21]:
cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [22]:
_ = cv2.drawContours(thresh_Countours, cnts, -1, (255, 0, 0), 3)

In [23]:
cv_imshow('thresh_contours', thresh_Countours)

In [24]:
# 此时检测到的轮廓很多，我们下面需要过滤， 选择出答案的那些圆形轮廓
questionCnts = []
for c in cnts:
    # 计算比例和大小
    (x, y, w, h) = cv2.boundingRect(c)   # 外接矩形， 原型的外接矩形的长宽比例接近1
    ar = w / float(h)
    
    # 根据实际情况指定标准
    if w >= 20 and h >= 20 and ar >= 0.0 and ar <= 1.1:
        questionCnts.append(c)

In [25]:
temp = warped.copy()
_ = cv2.drawContours(temp, questionCnts, -1, (255, 0, 0), 3)
cv_imshow('tmp', temp)

In [26]:
def sort_contours(cnts, method='left-to-right'):
    reverse = False
    i = 0
    if method == 'right-to-left' or method == 'bottom-to-top':
        reverse = True
    if method == 'top-to-bottom' or method == 'bottom-to-top':
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes), key=lambda b: b[1][i], reverse=reverse))
    return cnts, boundingBoxes

In [27]:
# 接下来， 就是把这些圆圈排序，首先需要先按照每个题排列好，不同题的x坐标一致， y坐标是从小到大
questionCnts = sort_contours(questionCnts, method='top-to-bottom')[0]

## 遍历检测

In [41]:
# 正确答案
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

correct = 0

In [46]:
# 遍历每个题目
for (q_idx, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    # 拿到当前题目的5个选项，并且从左到右排序
    cnts = sort_contours(questionCnts[i:i+5])[0]
    selected = None
    
    # 遍历每个结果
    for (j, c) in enumerate(cnts):
        # 使用mask来判断选择的是哪个答案
        mask = np.zeros(thresh.shape, dtype='uint8')
        cv2.drawContours(mask, [c], -1, 255, -1)
        
        # 通过计算非零点数量来算是否选择了当前答案
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total_non_zeros = cv2.countNonZero(mask)
        
        # 通过阈值判断  选出非零点数量最多的那个来
        if selected is None or total_non_zeros > selected[0]:
            selected = (total_non_zeros, j)
    
    # 对比正确答案
    color = (255, 0, 0)
    true_ans = ANSWER_KEY[q_idx]
    
    # 选择正确
    if true_ans == selected[1]:
        correct += 1
        color = (0, 255, 0)
    
    # 绘图
    cv2.drawContours(warped, [cnts[true_ans]], -1, color, 3)

In [47]:
cv_imshow('warped', warped)

In [129]:
len(questionCnts)

25

In [49]:
exam_img = warped.copy()
score = (correct / 5) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(exam_img, "{:.2f}%".format(score), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv_imshow("Original", img)
cv_imshow("Exam", exam_img)

[INFO] score: 80.00%


## 封装成算法检测

In [1]:
from answer_sheet import answer_dect
from utils import cv_imshow
import cv2

In [2]:
test_img = cv2.imread('images/test_02.png')
# 正确答案
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

exam_img = answer_dect(test_img, ANSWER_KEY)

cv_imshow('test_img', test_img)
cv_imshow('exam_img', exam_img)